# Simulate a feedback survey
This notebook provides example code for simulating a feedback survey with AI agents in [EDSL](https://github.com/expectedparrot/edsl), an open-source library for simulating surveys, experiments and research tasks with AI agents and large language models. We also show how EDSL can be used to generate content by first using it to prompt a model to draft the content for the feedback survey (names for a yoga studio) and the personas that we use for the AI agents (target customers of the studio).

Before running the code below please see instructions for [getting started](https://www.expectedparrot.com/getting-started) using EDSL, and tips and tutorials at our [documentation page](https://docs.expectedparrot.com).

## Create a question

We start by creating a question to prompt a language model to suggest some names for a yoga studio.
EDSL comes with many common [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model - multiple choice, free text, linear scale, etc. 
Here we use `QuestionList` to prompt a model to return a list of items:

In [1]:
from edsl import QuestionList

q = QuestionList(
    question_name = "yoga_studio_name",
    question_text = "What are some creative names for a yoga studio?",
    max_list_items = 10
)

## Select some models to answer the question
EDSL works with many popular [language models that we can select](https://docs.expectedparrot.com/en/latest/language_models.html) to generate responses to questions.
A list of current available models and pricing and performance information is available [here](https://www.expectedparrot.com/models).

To see a list of all service providers:

In [2]:
from edsl import ModelList, Model

Model.services()

PrettyList([['anthropic'],
            ['azure'],
            ['bedrock'],
            ['deep_infra'],
            ['deepseek'],
            ['google'],
            ['groq'],
            ['mistral'],
            ['ollama'],
            ['openai'],
            ['openai_v2'],
            ['perplexity'],
            ['together'],
            ['xai']])

To select models to use with a question or survey:

In [3]:
models = ModelList([
    Model("claude-3-sonnet-20240229", service_name = "anthropic"),
    Model("gpt-4o", service_name = "openai")
])

## Run the question 
We administer a question to a language model by calling the `run()` method on it.
This generates a formatted dataset of `Results`:

In [4]:
results = q.by(models).run()

## Inspect the results
EDSL comes with built-in [methods for analyzing results](https://docs.expectedparrot.com/en/latest/results.html).
Here we inspect the responses for each model:

In [5]:
results.select("model", "yoga_studio_name")

Dataset([{'model.model': ['claude-3-sonnet-20240229', 'gpt-4o']}, {'answer.yoga_studio_name': [['Zen Oasis', 'Mindful Movements', 'Serenity Studio', 'Breathe Bliss', 'Harmony Haven', 'Peaceful Postures', 'Tranquil Tribe', 'Soul Stretch', 'Chakra Chimes', 'Yogic Vibes'], ['Zen Flow', 'Soul Stretch', 'Namaste Nest', 'Peaceful Poses', 'Harmony Haven', 'Serenity Studio', 'Mindful Movements', 'Tranquil Transitions', 'Blissful Balance', 'Inner Light Yoga']]}])

## Use the responses in new questions
We can format the responses to use them as options to new questions:

In [6]:
claude_names = results.filter("model.model == 'claude-3-sonnet-20240229'").select("yoga_studio_name").to_list()[0]
claude_names

['Zen Oasis',
 'Mindful Movements',
 'Serenity Studio',
 'Breathe Bliss',
 'Harmony Haven',
 'Peaceful Postures',
 'Tranquil Tribe',
 'Soul Stretch',
 'Chakra Chimes',
 'Yogic Vibes']

In [7]:
gpt4o_names = results.filter("model.model == 'gpt-4o'").select("yoga_studio_name").to_list()[0]
gpt4o_names

['Zen Flow',
 'Soul Stretch',
 'Namaste Nest',
 'Peaceful Poses',
 'Harmony Haven',
 'Serenity Studio',
 'Mindful Movements',
 'Tranquil Transitions',
 'Blissful Balance',
 'Inner Light Yoga']

We can optionally randomize the list of options:

In [8]:
studio_names = list(set(gpt4o_names + claude_names))

In [9]:
import random

random.shuffle(studio_names)

## Constructing a survey
Here we create some new questions in different types and combine them into a survey to administer them together:

In [10]:
from edsl import QuestionMultipleChoice, QuestionCheckBox, Survey

q1 = QuestionMultipleChoice(
    question_name = "favorite",
    question_text = "What is your favorite name for a yoga studio?",
    question_options = studio_names
)

q2 = QuestionCheckBox(
    question_name = "pick",
    question_text = "Pick the 3 best names for a yoga studio.",
    question_options = studio_names,
    min_selections = 3,
    max_selections = 3
)

survey = Survey([q1, q2])

survey

Survey(questions=[Question('multiple_choice', question_name = """favorite""", question_text = """What is your favorite name for a yoga studio?""", question_options = ['Peaceful Poses', 'Zen Flow', 'Soul Stretch', 'Zen Oasis', 'Tranquil Transitions', 'Namaste Nest', 'Serenity Studio', 'Tranquil Tribe', 'Inner Light Yoga', 'Mindful Movements', 'Yogic Vibes', 'Harmony Haven', 'Peaceful Postures', 'Chakra Chimes', 'Blissful Balance', 'Breathe Bliss']), Question('checkbox', question_name = """pick""", question_text = """Pick the 3 best names for a yoga studio.""", min_selections = 3, max_selections = 3, question_options = ['Peaceful Poses', 'Zen Flow', 'Soul Stretch', 'Zen Oasis', 'Tranquil Transitions', 'Namaste Nest', 'Serenity Studio', 'Tranquil Tribe', 'Inner Light Yoga', 'Mindful Movements', 'Yogic Vibes', 'Harmony Haven', 'Peaceful Postures', 'Chakra Chimes', 'Blissful Balance', 'Breathe Bliss'])], memory_plan={}, rule_collection=RuleCollection(rules=[Rule(current_q=0, expression="True", next_q=1, priority=-1, question_name_to_index={'favorite': 0}, before_rule=False), Rule(current_q=1, expression="True", next_q=2, priority=-1, question_name_to_index={'favorite': 0, 'pick': 1}, before_rule=False)], num_questions=2), question_groups={}, questions_to_randomize=[])

## Designing agents to answer the survey
Next we can design AI agents with relevant traits to answer the questions.
Here we use a model to draft some personas, and then create "agent" objects for them to use with the survey:

In [11]:
q = QuestionList(
    question_name = "personas",
    question_text = "Draft 5 diverse personas for patrons of a yoga studio."
)

In [12]:
personas = q.run().select("personas").to_list()[0]
personas

['A busy corporate executive seeking stress relief and work-life balance',
 'A retired senior looking to maintain flexibility and social connections',
 'A college student exploring mindfulness and physical fitness',
 'A new parent using yoga to regain strength and mental clarity',
 'An athlete incorporating yoga for cross-training and injury prevention']

In [13]:
from edsl import AgentList, Agent

agents = AgentList(
    Agent(traits = {"persona":p}) for p in personas
)

agents

AgentList([Agent(traits = {'persona': 'A busy corporate executive seeking stress relief and work-life balance'}), Agent(traits = {'persona': 'A retired senior looking to maintain flexibility and social connections'}), Agent(traits = {'persona': 'A college student exploring mindfulness and physical fitness'}), Agent(traits = {'persona': 'A new parent using yoga to regain strength and mental clarity'}), Agent(traits = {'persona': 'An athlete incorporating yoga for cross-training and injury prevention'})])

### Run the survey with the agents
Here we add the agents to the survey and run it with the language models we selected above (to compare responses):

In [14]:
results = survey.by(agents).by(models).run()

In [15]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "favorite", "pick")
)

Dataset([{'model.model': ['claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o']}, {'agent.persona': ['A busy corporate executive seeking stress relief and work-life balance', 'A college student exploring mindfulness and physical fitness', 'A new parent using yoga to regain strength and mental clarity', 'A retired senior looking to maintain flexibility and social connections', 'An athlete incorporating yoga for cross-training and injury prevention', 'A busy corporate executive seeking stress relief and work-life balance', 'A college student exploring mindfulness and physical fitness', 'A new parent using yoga to regain strength and mental clarity', 'A retired senior looking to maintain flexibility and social connections', 'An athlete incorporating yoga for cross-training and injury prevention']}, {'answer.favorite': ['Zen Oasis', 'Zen Oasis', 'Tranquil Tribe', 'Zen Oasis', 'Zen Oasis', 'Zen Flow', 'Zen Flow', 'Namaste Nest', 'Serenity Studio', 'Zen Flow']}, {'answer.pick': [['Zen Oasis', 'Tranquil Transitions', 'Serenity Studio'], ['Zen Oasis', 'Tranquil Transitions', 'Harmony Haven'], ['Zen Oasis', 'Tranquil Transitions', 'Breathe Bliss'], ['Zen Oasis', 'Serenity Studio', 'Harmony Haven'], ['Zen Oasis', 'Tranquil Transitions', 'Harmony Haven'], ['Zen Flow', 'Serenity Studio', 'Blissful Balance'], ['Zen Flow', 'Soul Stretch', 'Inner Light Yoga'], ['Zen Flow', 'Soul Stretch', 'Harmony Haven'], ['Zen Flow', 'Soul Stretch', 'Serenity Studio'], ['Zen Flow', 'Soul Stretch', 'Harmony Haven']]}])

We could also solicit feedback on individual names.
This can be done by creating a "scenario" of each question for each name:

In [16]:
from edsl import QuestionLinearScale, QuestionFreeText, Survey, ScenarioList

In [17]:
scenarios = ScenarioList.from_list("studio_name", studio_names)
scenarios.table()

studio_name
--------------------
Peaceful Poses
Zen Flow
Soul Stretch
Zen Oasis
Tranquil Transitions
Namaste Nest
Serenity Studio
Tranquil Tribe
Inner Light Yoga
Mindful Movements
Yogic Vibes
Harmony Haven
Peaceful Postures
Chakra Chimes
Blissful Balance
Breathe Bliss

In [18]:
q_memorable = QuestionLinearScale(
    question_name = "memorable",
    question_text = "How memorable is this yoga studio name: {{ scenario.studio_name }}",
    question_options = [1,2,3,4,5],
    option_labels = {1:"Not at all memorable", 5:"Very memorable"}
)

q_criticism = QuestionFreeText(
    question_name = "criticism",
    question_text = "What are some criticisms people might have about this yoga studio name: {{ scenario.studio_name }}"
)

survey = Survey([q_memorable, q_criticism])

results = survey.by(scenarios).by(agents).by(models).run()

In [19]:
(
    results
    .sort_by("model", "persona")
    .select("model", "persona", "studio_name", "memorable", "criticism")
)

Dataset([{'model.model': ['claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'claude-3-sonnet-20240229', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o', 'gpt-4o']}, {'agent.persona': ['A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executive seeking stress relief and work-life balance', 'A busy corporate executi

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook to share with others (visibility can be *public*, *private* or *unlisted* by default):

In [20]:
# from edsl import Notebook

# nb = Notebook(path = "yoga_studio_name_survey.ipynb")

# nb.push(
#     description = "Feedback survey", 
#     alias = "feedback-survey-notebook",
#     visibility = "public"
# )

To update an object at Coop:

In [ ]:
from edsl import Notebook

nb = Notebook(path = "yoga_studio_name_survey.ipynb") # resave

nb.patch("https://www.expectedparrot.com/content/RobinHorton/feedback-survey-notebook", value = nb)